# Graph Traversal

* Breath-First Search (BFS)
* Depth-First Search (DFS)
* Topological Sort

In [139]:
import random
import numpy as np

from collections import deque, defaultdict, Counter

In [5]:
class Node(object):
    
    def __init__(self, name, color, d, pi):
        self.name = name
        self.color = color
        self.d = d
        self.pi = pi
    
    def __repr__(self):
        return 'Node(name=%s, color=%s, distance=%i, parent=%s)' % \
               (self.name,self.color,self.d,self.pi)
        
class Node_DFS(Node):
    # this version allows adding time stamps (parathetical structure for DFS)
    
    def __init__(self, name, color, d, f, pi):
        super(Node_DFS, self).__init__(name,color,d,pi)
        self.f = f
    
    def __repr__(self):
        return 'Node(name=%s, color=%s, begin=%i, end=%i, parent=%s)' % \
               (self.name,self.color,self.d,self.f,self.pi) 

### BFS

In [53]:
#  r--s  t--u
#  |  | /| /|
#  v  w--x--y

adj_list = {'r': ['s','v'], 's': ['r','w'], 't': ['w','x','u'], 'u': ['t','x','y'],
            'v': ['r'], 'w': ['s','t','x'], 'x': ['w','t','u','y'], 'y': ['x','u']}
adj_list.keys()

['s', 'r', 'u', 't', 'w', 'v', 'y', 'x']

In [59]:
# Explore all nodes

def BFS(adj_list, root, verbose=False):
    G = {}
    for u in filter(lambda x:x!=root, adj_list.keys()):
        G[u] = Node(u,'WHITE',0,None)
    G[root] = Node(root,'GRAY',0,None)
    Q = deque()
    Q.append(root)
    count = 0
    while len(Q)>0:
        if verbose:
            print 'At %dth: %s | Q: %s' % (count, u, Q)
        u = Q.popleft()
        count += 1
        for v in adj_list[u]:
            if G[v].color == 'WHITE':
                G[v].color = 'GRAY'
                G[v].d = G[u].d + 1
                G[v].pi = u
                Q.append(v)
        G[u].color = 'BLACK'
    return G

In [60]:
BFS(adj_list, 's', verbose=1)

At 0th: x | Q: deque(['s'])
At 1th: s | Q: deque(['r', 'w'])
At 2th: r | Q: deque(['w', 'v'])
At 3th: w | Q: deque(['v', 't', 'x'])
At 4th: v | Q: deque(['t', 'x'])
At 5th: t | Q: deque(['x', 'u'])
At 6th: x | Q: deque(['u', 'y'])
At 7th: u | Q: deque(['y'])


{'r': Node(name=r, color=BLACK, distance=1, parent=s),
 's': Node(name=s, color=BLACK, distance=0, parent=None),
 't': Node(name=t, color=BLACK, distance=2, parent=w),
 'u': Node(name=u, color=BLACK, distance=3, parent=t),
 'v': Node(name=v, color=BLACK, distance=2, parent=r),
 'w': Node(name=w, color=BLACK, distance=1, parent=s),
 'x': Node(name=x, color=BLACK, distance=2, parent=w),
 'y': Node(name=y, color=BLACK, distance=3, parent=x)}

In [63]:
# shortest path

def shortest_path(adj_list, u, v):
    G = BFS(adj_list, u)
    print 'The shortest path from %s to %s is %d' % \
          (u, v, G[v].d)

In [64]:
shortest_path(adj_list, 'r', 't')

The shortest path from r to t is 3


In [81]:
# connectivity

#  1-3             6
#  \/             /\
#  5      2-4    8-10
#  /\
# 7-9

disconnected_adj_list = {'1':['3','5'],'2':['4'],'3':['1','5'],'4':['2'],
                         '5':['1','3','7','9'],'6':['8','10'],'7':['5','9'],
                         '8':['6','10'],'9':['5','7'],'10':['6','8']}

def get_explored(G):
    return [name for name,node in G.iteritems() if node.color=='BLACK']

def connected_groups(adj_list, verbose=False):
    explored, groups = [], []
    for v in adj_list.iterkeys():
        if v not in explored:
            cur_explored = get_explored(BFS(adj_list, v))
            groups.append(cur_explored)
            explored.extend(cur_explored)
            if verbose:
                print 'Explored: %s | Got group: %s' % \
                      (v, str(cur_explored))
    assert all(v in adj_list for v in explored)
    return groups

In [82]:
connected_groups(disconnected_adj_list, verbose=1)

Explored: 1 | Got group: ['1', '3', '5', '7', '9']
Explored: 10 | Got group: ['10', '6', '8']
Explored: 2 | Got group: ['2', '4']


[['1', '3', '5', '7', '9'], ['10', '6', '8'], ['2', '4']]

### DFS

In [83]:
# u-*v  w
# |/*| /|
# *  ** *
# x*-y  z>*

adj_list = {'u': ['x','v'], 'v': ['y'], 'w': ['y','z'], 'x': ['v'], 
            'y': ['x'], 'z': ['z']}
adj_list.keys()

['u', 'w', 'v', 'y', 'x', 'z']

##### VERSION 1: RECURSION-BASED

In [100]:
tm = 0 # global time, reset before each run

def DFS_visit(adj_list, G, u):
    global tm
    tm += 1
    G[u].d = tm
    G[u].color = 'GRAY'
    for v in adj_list[u]:
        if G[v].color=='WHITE':
            G[v].pi = u
            DFS_visit(adj_list,G,v)
    G[u].color = 'BLACK'
    tm += 1
    G[u].f = tm

def DFS(adj_list):
    G = {}
    for u in adj_list.keys():
        G[u] = Node_DFS(u,'WHITE',0,0,None)
    global tm
    tm = 0
    for u in adj_list.keys():
        if G[u].color=='WHITE':
            DFS_visit(adj_list, G,u)
    return G

In [101]:
DFS(adj_list)

{'u': Node(name=u, color=BLACK, begin=1, end=8, parent=None),
 'v': Node(name=v, color=BLACK, begin=3, end=6, parent=x),
 'w': Node(name=w, color=BLACK, begin=9, end=12, parent=None),
 'x': Node(name=x, color=BLACK, begin=2, end=7, parent=u),
 'y': Node(name=y, color=BLACK, begin=4, end=5, parent=v),
 'z': Node(name=z, color=BLACK, begin=10, end=11, parent=w)}

##### VERSION 2: STACK

In [105]:
#   a-c-e
#  /| |/
# s-b-d

adj_list = {'s': ['a','b'], 'a': ['s','b','c'], 'c': ['a','d','e'],
            'e': ['c','d'], 'd': ['c','e','b'], 'b': ['s','a','d']}
adj_list.keys()

['a', 'c', 'b', 'e', 'd', 's']

In [131]:
def not_visited(G, v_list):
    return [v for v in v_list if G[v].color=='WHITE']

def DSF_stack(adj_list, root, verbose=False):
    stack = list()
    G = {}
    for u in adj_list.keys():
        G[u] = Node(u,'WHITE',0,None)
    stack.append(root)
    G[root].color = 'BLACK'
    prev = root
    while len(stack)>0:
        u = stack[-1]
        v_list = adj_list[u]
        to_visit = not_visited(G, v_list)
        if len(to_visit)==0: 
            stack.pop()
            continue
        v = to_visit[0]
        if verbose:
            print 'visiting:', v
        G[v].color = 'BLACK'
        G[v].pi = prev
        stack.append(v)
        prev = v
    return G

In [130]:
DSF_stack(adj_list, 's', verbose=1)

visiting: a
visiting: b
visiting: d
visiting: c
visiting: e


{'a': Node(name=a, color=BLACK, distance=0, parent=s),
 'b': Node(name=b, color=BLACK, distance=0, parent=a),
 'c': Node(name=c, color=BLACK, distance=0, parent=d),
 'd': Node(name=d, color=BLACK, distance=0, parent=b),
 'e': Node(name=e, color=BLACK, distance=0, parent=c),
 's': Node(name=s, color=BLACK, distance=0, parent=None)}

### Topological Sort

* DEF: a _topological sort_ of a dag $G=(V,E)$ is a linear ordering of all its vertices such that if $G$ contains an edge $(u,v)$, then $u$ appears before $v$ in the ordering. If the graph contains a cycle, then no linear ordering is possible. (CLRS, p612)

In [152]:
adj_list = {'undershorts':['pants','shoes'], 'socks':['shoes'],
            'watch':[], 'pants':['shoes','belt'], 'shoes':[],
            'shirt':['belt','tie'], 'belt':['jacket'], 'tie':['jacket'], 'jacket':[]}
adj_list.keys()

['jacket',
 'shoes',
 'tie',
 'undershorts',
 'shirt',
 'belt',
 'watch',
 'socks',
 'pants']

##### VERSION 1: TIME STAMPS

In [153]:
tm = 0 # global time, reset before each run
ordering = []

def DFS_visit_top(adj_list, G, u, verbose=False):
    global tm, ordering
    tm += 1
    G[u].d = tm
    G[u].color = 'GRAY'
    for v in adj_list[u]:
        if G[v].color=='WHITE':
            G[v].pi = u
            DFS_visit_top(adj_list,G,v,verbose)
    G[u].color = 'BLACK'
    tm += 1
    G[u].f = tm
    ordering.insert(0, (u,str(G[u].d)+'/'+str(G[u].f)))
    if verbose:
        print 'Current ordering:', ordering

def DFS(adj_list, verbose=False):
    G = {}
    for u in adj_list.keys():
        G[u] = Node_DFS(u,'WHITE',0,0,None)
    global tm
    tm = 0
    for u in adj_list.keys():
        if G[u].color=='WHITE':
            DFS_visit_top(adj_list,G,u,verbose)
    return G

In [154]:
DFS(adj_list, verbose=1)

Current ordering: [('jacket', '1/2')]
Current ordering: [('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('undershorts', '7/12'), ('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('shirt', '13/14'), ('undershorts', '7/12'), ('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('watch', '15/16'), ('shirt', '13/14'), ('undershorts', '7/12'), ('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]
Current ordering: [('socks', '17/18'), ('watch', '15/16'), ('shirt', '13/14'), ('undershorts', '7/12'), ('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4')

{'belt': Node(name=belt, color=BLACK, begin=9, end=10, parent=pants),
 'jacket': Node(name=jacket, color=BLACK, begin=1, end=2, parent=None),
 'pants': Node(name=pants, color=BLACK, begin=8, end=11, parent=undershorts),
 'shirt': Node(name=shirt, color=BLACK, begin=13, end=14, parent=None),
 'shoes': Node(name=shoes, color=BLACK, begin=3, end=4, parent=None),
 'socks': Node(name=socks, color=BLACK, begin=17, end=18, parent=None),
 'tie': Node(name=tie, color=BLACK, begin=5, end=6, parent=None),
 'undershorts': Node(name=undershorts, color=BLACK, begin=7, end=12, parent=None),
 'watch': Node(name=watch, color=BLACK, begin=15, end=16, parent=None)}

In [155]:
print 'Final ordering:', ordering

Final ordering: [('socks', '17/18'), ('watch', '15/16'), ('shirt', '13/14'), ('undershorts', '7/12'), ('pants', '8/11'), ('belt', '9/10'), ('tie', '5/6'), ('shoes', '3/4'), ('jacket', '1/2')]


##### VERSION 2: LABELINGS

In [162]:
adj_list = {'undershorts':['pants','shoes'], 'socks':['shoes'],
            'watch':[], 'pants':['shoes','belt'], 'shoes':[],
            'shirt':['belt','tie'], 'belt':['jacket'], 'tie':['jacket'], 'jacket':[]}
current_label = len(adj_list)

def DFS_loop():
    global adj_list
    G = {}
    for u in adj_list.keys():
        G[u] = Node(u,'WHITE',0,None)
    for u in adj_list:
        if G[u].color=='WHITE':
            DFS(G, u)
    return G

def DFS(G, u):
    global adj_list, current_label
    G[u].color = 'BLACK'
    for v in adj_list[u]:
        if G[v].color=='WHITE':
            DFS(G, v)
    G[u].d = current_label
    current_label -= 1

In [163]:
sorted([(u,node.d)for u,node in DFS_loop().iteritems()], key=lambda x:x[1])

[('socks', 1),
 ('watch', 2),
 ('shirt', 3),
 ('undershorts', 4),
 ('pants', 5),
 ('belt', 6),
 ('tie', 7),
 ('shoes', 8),
 ('jacket', 9)]

### Strongly-Connected Components (SCC)

* A subgraph $C$ of $G$ is an SCC, if 
    * All the nodes in $C$ are connected, and
    * $\forall v_i,v_j \in C$, if $(v_i,v_j)$ is an edge, $(v_j,v_i)$ is an edge too.

In [131]:
test0_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test0.txt' # 33300
test1_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test1.txt' # 33300
test2_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test2.txt' # 33200
test3_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test3.txt' # 33110
test4_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test4.txt' # 71000
test5_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC_test5.txt' # 63210

In [155]:
def read_graph(path):
    G = defaultdict(lambda: [[],[]]) # first sublist: points to; second sublist: being pointed to
    with open(path) as source:
        for line in source:
            u,v = line.split()
            u,v = int(u),int(v)
            G[u][0].append(v) # u points to v
            G[v][1].append(u) # v being pointed to by u
    return G

def rev_pass(G):
    f = {}
    t = 0
    stack = []
    visited = set()
    for cur_node in xrange(1, len(G)+1):
        if cur_node not in visited:
            stack.append(cur_node)
            while stack:
                u = stack[-1] # set the 1st on stack as base
                visited.add(u)
                if G[u][1]==[]: # i.e. no more targets in G_rev
                    t += 1
                    f[t] = u
                    stack.pop() # done with u
                else:
                    for v in G[u][1]:
                        if v not in visited:
                            visited.add(v)
                            stack.append(v)
                    G[u][1] = [] # empty targets in G_rev to mark termination
    return f

def fwd_pass(G, f):
    leader2comp = Counter()
    stack = []
    visited = set()
    for i in xrange(len(f),0,-1):
        cur_node = f[i]
        if cur_node not in visited:
            stack.append(cur_node)
            leader2comp[cur_node] += 1
            while stack:
                u = stack[-1] # set the 1st on stack as base
                visited.add(u)
                if G[u][0]==[]: # no more targets in G
                    stack.pop() # done with u
                else:
                    for v in G[u][0]:
                        if v not in visited:
                            visited.add(v)
                            stack.append(v)
                            leader2comp[cur_node] += 1 # place unexplored node under the group of cur_node
                    G[u][0] = [] # empty targets in G to mark termination
    return leader2comp

def kosaraju(path, topk=10):
    G = read_graph(path)
    f = rev_pass(G)
    leader2comp = fwd_pass(G,f)
    print leader2comp.most_common(topk)

In [156]:
kosaraju(test0_path)
kosaraju(test1_path)
kosaraju(test2_path)
kosaraju(test3_path)
kosaraju(test4_path)
kosaraju(test5_path)

[(8, 3), (1, 3), (9, 3)]
[(8, 3), (1, 3), (9, 3)]
[(1, 3), (6, 3), (4, 2)]
[(1, 3), (6, 3), (4, 1), (5, 1)]
[(1, 7), (5, 1)]
[(7, 6), (2, 3), (3, 2), (1, 1)]


In [157]:
graph_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/SCC.txt'
kosaraju(graph_path)

[(1, 434821), (498116, 968), (13934, 459), (33604, 313), (97128, 211), (18994, 205), (4479, 197), (448227, 177), (80272, 162), (135140, 152)]
